In [1]:
!pip install datasets evaluate transformers corus razdel seqeval transformers[torch] --quiet
!pip install accelerate -U --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 18.8 MB/s eta 0:00:00


# **Задание**
1.  Дообучить берт на задачу NER
2.  Дообучить GPT на генерацию текста
3. *Дообучить T5 на задачу суммаризации текста


## Импорты

In [17]:
!wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
!pip install corus razdel

--2023-12-11 09:44:56--  https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json [following]
--2023-12-11 09:44:56--  https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1773014 (1.7M) [text/plain]
Saving to: ‘rudrec_annotated.json’

rudrec_annotated.js 100%[===================>]   1.69M  --.-KB/s    in 0.07s   

2023-12-11 09:44:56 (25.6 MB/s) - ‘rudrec_annotated.json’ saved [1773014/1773014]



In [18]:
import numpy as np
import pandas as pd
import re

import torch
from datasets import Dataset, DatasetDict, load_dataset
from evaluate import load as load_metric

from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForTokenClassification, AutoModelForCausalLM, T5ForConditionalGeneration
from transformers import DataCollatorForTokenClassification
from transformers import TextDataset, DataCollatorForLanguageModeling

from corus import load_rudrec
from razdel import tokenize

from sklearn.model_selection import train_test_split

## Настройки

In [32]:
!wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json

--2023-12-11 09:56:27--  https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json [following]
--2023-12-11 09:56:27--  https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1773014 (1.7M) [text/plain]
Saving to: ‘rudrec_annotated.json.1’

rudrec_annotated.js 100%[===================>]   1.69M  --.-KB/s    in 0.07s   

2023-12-11 09:56:27 (25.0 MB/s) - ‘rudrec_annotated.json.1’ saved [1773014/1773014]



In [57]:
!wget https://drive.google.com/file/d/1ZSCK9EyorylwspREqBWjjyeuSgdzvhbg/view?usp=drive_link

--2023-12-11 10:38:18--  https://drive.google.com/file/d/1ZSCK9EyorylwspREqBWjjyeuSgdzvhbg/view?usp=drive_link
Resolving drive.google.com (drive.google.com)... 74.125.202.100, 74.125.202.138, 74.125.202.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.202.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=drive_link’

view?usp=drive_link     [ <=>                ]  82.73K  --.-KB/s    in 0.001s  

2023-12-11 10:38:18 (105 MB/s) - ‘view?usp=drive_link’ saved [84719]



In [60]:
files.upload()

Saving all_recepies_inter.csv to all_recepies_inter.csv


In [61]:
BERT_MODEL = 'cointegrated/rubert-tiny'
BERT_DATA = 'rudrec_annotated.json'
BERT_BATCH_SIZE = 16
BERT_EPOCHS = 10

GPT_MODEL = 'sberbank-ai/rugpt3small_based_on_gpt2'
GPT_DATA = 'all_recepies_inter.csv'
GPT_BATCH_SIZE = 16
GPT_EPOCHS = 3

T5_MODEL = 'IlyaGusev/rut5_base_sum_gazeta'
T5_DATA = 'IlyaGusev/gazeta'
T5_BATCH_SIZE = 16
T5_EPOCHS = 5

device = 'cuda'

## 1. Дообучить берт на задачу NER

### Данные

In [34]:
def extract_labels(item):
    raw_toks = list(tokenize(item.text))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item.text)
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    for e in item.entities:
        e_words = sorted({idx for idx in char2word[e.start:e.end] if idx is not None})
        word_labels[e_words[0]] = 'B-' + e.entity_type
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + e.entity_type

    return {'tokens': words, 'tags': word_labels}

In [36]:
data = [extract_labels(item) for item in list(load_rudrec(BERT_DATA))]

data_train, data_test = train_test_split(data, test_size=0.1)
pd.DataFrame(data_train).head()

,tokens,tags
0,"[Не, буду, конкретизировать, этот, препарат, д...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-Find..."
1,"[Да, ,, сонливость, ,, вялость, ,, апатия, -, ...","[O, O, B-ADR, O, B-ADR, O, B-ADR, O, O, O, O, ..."
2,"[Ликопид, стимулировал, имунную, систему, даже...","[B-Drugname, O, O, O, O, O, O, O, O, O]"
3,"[Итог, :, препарат, работает, ,, но, такая, ра...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,"[Хорошо, помогает, при, первых, симптомах, про...","[O, O, O, O, O, B-DI, O, B-DI, O]"


In [37]:
label_list = sorted({label for item in data_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list

label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

In [38]:
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL)

def tokenize(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"],
                                 truncation=True,
                                 padding=True,
                                 is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [39]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(data_train)),
    'test': Dataset.from_pandas(pd.DataFrame(data_test))
})
tokenized_data = ner_data.map(tokenize, batched=True)

Map:   0%|          | 0/4328 [00:00<?, ? examples/s]

Map:   0%|          | 0/481 [00:00<?, ? examples/s]

### Модель

In [40]:
model = AutoModelForTokenClassification.from_pretrained(BERT_MODEL, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
for param in model.parameters():
    param.requires_grad = True

In [42]:
metric = load_metric("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [43]:
model = model.to(device)
tokenized_data = tokenized_data.with_format("torch", device=device)

args = TrainingArguments("ner",
                         evaluation_strategy = "epoch",
                         learning_rate=2e-5,
                         per_device_train_batch_size=BERT_BATCH_SIZE,
                         per_device_eval_batch_size=BERT_BATCH_SIZE,
                         num_train_epochs=BERT_EPOCHS,
                         weight_decay=0.01,
                         save_strategy='no',
                         report_to='none')

data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(model,
                  args,
                  train_dataset=tokenized_data["train"],
                  eval_dataset=tokenized_data["test"],
                  data_collator=data_collator,
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics)

### Обучение

In [44]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.553217,0.575875,0.273567,0.370927,0.866257
2,0.704800,0.464575,0.538728,0.479975,0.507657,0.883149
3,0.704800,0.399770,0.605479,0.544670,0.573467,0.896843
4,0.412200,0.377299,0.598810,0.619840,0.609143,0.903813
5,0.412200,0.359392,0.606061,0.640789,0.622941,0.906437
6,0.341600,0.342060,0.632239,0.652495,0.642207,0.910291
7,0.341600,0.334442,0.624490,0.659889,0.641702,0.910291
8,0.304500,0.332957,0.618857,0.667283,0.642158,0.910947
9,0.304500,0.328908,0.618286,0.666667,0.641565,0.910701
10,0.293600,0.326297,0.627394,0.666051,0.646145,0.911603


TrainOutput(global_step=2710, training_loss=0.4009947041303909, metrics={'train_runtime': 129.9892, 'train_samples_per_second': 332.951, 'train_steps_per_second': 20.848, 'total_flos': 130996560996000.0, 'train_loss': 0.4009947041303909, 'epoch': 10.0})

In [45]:
trainer.evaluate()

{'eval_loss': 0.3262966275215149,
 'eval_precision': 0.6273940800928612,
 'eval_recall': 0.6660505237215034,
 'eval_f1': 0.6461446503287507,
 'eval_accuracy': 0.9116031160311603,
 'eval_runtime': 1.0153,
 'eval_samples_per_second': 473.751,
 'eval_steps_per_second': 30.533,
 'epoch': 10.0}

In [47]:
text = ' '.join(ner_data['test'][20]['tokens'])
tokens = tokenizer(text, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}
with torch.no_grad():
    pred = model(**tokens)

indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])

print(text)
for t, idx in zip(token_text, indices):
    print(f'{t:15s} {label_list[idx]:10s}')

Но , если более сильный стресс , то они не справятся с ним .
[CLS]           O         
Но              O         
,               O         
если            O         
более           O         
сил             O         
##ьный          O         
стр             B-DI      
##есс           B-DI      
,               O         
то              O         
они             O         
не              O         
справ           O         
##ятся          O         
с               O         
ним             O         
.               O         
[SEP]           O         


## 2.  Дообучить GPT на генерацию текста

### Данные

In [62]:
data = pd.read_csv(GPT_DATA, sep='\t')
data.head()

,Unnamed: 0,name,composition,cooking_type,Инструкции,dish_type,Дата,photo,source,composition_inter
0,0,рассольник классический с перловкой и солеными...,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ...","варка,жарка",Подготовить указанные ингредиенты для приготов...,первое,05.06.2015,photo_1000menu_1.jpg,https://1000.menu/cooking/33395-rassolnik-s-pe...,"[{'product_id': 4253, 'name_source': 'Перловая..."
1,1,Суп пюре из белокочаной капусты,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'},...",варка,"Необходимые ингредиенты\r\nНарезаем лук, морко...",первое,27.06.2015,photo_1000menu_2.jpg,https://1000.menu/cooking/25399-sup-pure-iz-be...,"[{'product_id': 2286, 'name_source': 'Капуста ..."
2,2,Постные щи из квашеной капусты,"[{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'...","варка,жарка,тушение","Честно признаюсь, у меня не было репы на момен...",первое,12.02.2013,photo_1000menu_3.jpg,https://1000.menu/cooking/5159-postnje-shchi,"[{'product_id': 0, 'name_source': 'Капуста ква..."
3,3,Тюря- простой суп быстро и вкусно,"[{'Квас': 0.2, 'unit': 'л'}, {'Лук репчатый': ...",сырое,"\r\nНачинаем мы приготовление тюри с того, что...",первое,02.03.2011,photo_1000menu_4.jpg,https://1000.menu/cooking/5085-turya,"[{'product_id': 0, 'name_source': 'Квас', 'uni..."
4,4,Фасолевый суп из красной фасоли,"[{'Вода': 0.3, 'unit': 'л'}, {'Картошка': 0.3,...",варка,Подготовить ингредиенты. Для приготовления суп...,первое,28.01.2013,photo_1000menu_5.jpg,https://1000.menu/cooking/38765-fasolevyi-sup-...,"[{'product_id': 828, 'name_source': 'Вода', 'u..."


In [63]:
def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

In [64]:
data = data.loc[:5000, 'Инструкции']
data_train, data_test = train_test_split(data, test_size=0.15)

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

build_text_files(data_train, train_path)
build_text_files(data_test, test_path)

In [65]:
tokenizer = AutoTokenizer.from_pretrained(GPT_MODEL)

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

In [66]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (857511 > 2048). Running this sequence through the model will result in indexing errors


### Модель

In [67]:
model = AutoModelForCausalLM.from_pretrained(GPT_MODEL)
model.to(device)

args = TrainingArguments(output_dir="./gpt2-chief",
                         overwrite_output_dir=True,
                         num_train_epochs=GPT_EPOCHS,
                         per_device_train_batch_size=GPT_BATCH_SIZE,
                         per_device_eval_batch_size=GPT_BATCH_SIZE,
                         eval_steps = 400,
                         save_steps=800,
                         warmup_steps=500)

trainer = Trainer(model=model,
                  args=args,
                  data_collator=data_collator,
                  train_dataset=train_dataset,
                  eval_dataset=test_dataset)

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

### Обучение

In [68]:
trainer.train()

Step,Training Loss
500,2.446700
1000,2.233800


TrainOutput(global_step=1257, training_loss=2.295281967855012, metrics={'train_runtime': 712.7637, 'train_samples_per_second': 28.196, 'train_steps_per_second': 1.764, 'total_flos': 1312796491776000.0, 'train_loss': 2.295281967855012, 'epoch': 3.0})

In [70]:
prefix = "берем свежие томаты"
tokens = tokenizer(prefix, return_tensors='pt').to(device)
size = tokens['input_ids'].shape[1]
res = model.generate(**tokens,
                     do_sample=False,
                     max_length=size+50,
                     repetition_penalty=5.,
                     temperature=0.5,
                     num_beams=10,)

decoded = tokenizer.decode(res[0])
result = decoded[len(prefix):]
print(prefix + result)

берем свежие томаты, очистим их от кожуры и мелко нарежем.  2. В сковороде разогреваем растительное масло с двух сторон до золотистого цвета. Добавляем в сковороду к помидорам нарезанные кубиками болгарский перец


## 3*. Дообучить T5 на задачу суммаризации текста

In [71]:
from datasets import load_dataset

### Данные

In [72]:
data_train = load_dataset(T5_DATA, revision="v1.0", split='train[:10%]')
data_test = load_dataset(T5_DATA, revision="v1.0", split='test[:10%]')
data_train

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/52400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5265 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'summary', 'title', 'date', 'url'],
    num_rows: 5240
})

In [73]:
def len_tok(text):
    return len(text.split())

max_len_sum = max(map(len_tok,data_train['summary']))
max_len_tl = max(map(len_tok,data_train['title']))

tokenizer = AutoTokenizer.from_pretrained(T5_MODEL)

def tokenize(batch):
    tokenized_input = tokenizer(batch['summary'], padding='max_length', truncation=True, max_length=max_len_sum)
    tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tl)
    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

data_train = data_train.map(tokenize, batched=True, batch_size=8)
data_test = data_test.map(tokenize, batched=True, batch_size=8)

data_train.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
data_test.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

tokenizer_config.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Map:   0%|          | 0/5240 [00:00<?, ? examples/s]

Map:   0%|          | 0/577 [00:00<?, ? examples/s]

### Модель

In [74]:
output_dir = 'gazeta/output'
model = T5ForConditionalGeneration.from_pretrained(T5_MODEL).to(device)

args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=T5_EPOCHS,
    per_device_train_batch_size=T5_BATCH_SIZE,
    per_device_eval_batch_size=T5_BATCH_SIZE,
    eval_accumulation_steps=1,
    prediction_loss_only=True,
    learning_rate=0.00001,
    evaluation_strategy='steps',
    save_steps=1000,
    save_total_limit=1,
    remove_unused_columns=True,
    run_name='run_gazeta',
    logging_steps=500,
    eval_steps=500,
    logging_first_step=False,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=data_train,
    eval_dataset=data_test
)


config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/977M [00:00<?, ?B/s]

### Обучение

In [75]:
trainer.train()

Step,Training Loss,Validation Loss
500,8.563200,2.924042
1000,4.303900,2.809912
1500,2.384700,2.729460


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=1640, training_loss=4.8458271119652725, metrics={'train_runtime': 736.3591, 'train_samples_per_second': 35.58, 'train_steps_per_second': 2.227, 'total_flos': 2608755379200000.0, 'train_loss': 4.8458271119652725, 'epoch': 5.0})

In [77]:
ID = 100
input_text = data_test['summary'][ID]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask,
        max_length=512,
        num_beams=7,
        temperature = 1.3,
        repetition_penalty=1,
        length_penalty=1,
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print(input_text)
print(pred)

Российские фрегаты «Адмирал Макаров» и «Адмирал Григорович» с крылатыми ракетами «Калибр-НК» проходят через турецкие проливы Босфор и Дарданеллы в направлении Средиземного моря. Ранее эти корабли уже участвовали в военной операции РФ в Сирии и теперь направляются туда на фоне обострившейся ситуации в Идлибе.
Российские корабли «Калибр-НК» проходят через Босфор
